In [1]:
# Import depdendencies
import shutil
import sagemaker
import os
import numpy as np
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

No handlers could be found for logger "sagemaker"


In [ ]:
## Example: S3 output upload
# !aws s3 cp images/test s3://tickercardiology-echocv-sagemaker/model-data/test --recursive
# !aws s3 cp images/train s3://tickercardiology-echocv-sagemaker/model-data/train --recursive

In [ ]:
## Example: test model on image
test_image = load_img("images/test/a2c/a2c-1_00.jpg", target_size=(224, 224))
test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
result = predictor.predict({'inputs_input': test_image_array})

print(result)

In [3]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [2]:
# Set S3 address to training data
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
training_data_uri = "s3://{}/{}/train/".format(bucket, key)
# Set S3 address to validation data
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
validation_data_uri = "s3://{}/{}/validation/".format(bucket, key)
# Set S3 address to test data
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
testing_data_uri = "s3://{}/{}/test/".format(bucket, key)

In [4]:
# Display training script
!pygmentize 'keras_model_fn.py'

import tensorflow as tf
import argparse
import os
import numpy as np
import json
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.losses import categorical_crossentropy  

import numpy as np

INPUT_TENSOR_NAME = "inputs_input" # Watch out, it needs to match the name of the first layer + "_input"
BATCH_SIZE = 64
HEIGHT = 224
WIDTH = 224
DEPTH = 3

def model_compile(learning_rate):
    model = Sequential()
    model.add(Conv2D(input_shape=(HEIGHT,WIDTH,DEPTH),filters=64,kernel_size=(3,3),padding="same",activation="relu",name="inputs"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activati

In [17]:
# Create TensorFlow2 estimator 
# bucket = "sagemaker-ap-southeast-2-611188727347"
# key = "test1"
# model_path = "s3://{}/{}/".format(bucket, key)

estimator2 = TensorFlow(entry_point='keras_model_fn.py',
                             role=role,
                             train_instance_count=1,
#                              train_instance_type='ml.m5.2xlarge', #$0.672
                             train_instance_type='ml.g4dn.xlarge', #$0.958
                             framework_version='2.1.0',
                             py_version='py2',
                             hyperparameters={'epochs': 1},
#                              model_dir = model_path,
                             distributions={'parameter_server': {'enabled': True}})

In [ ]:
# Start training model

estimator2.fit(training_data_uri)

2020-11-05 05:19:58 Starting - Starting the training job...
2020-11-05 05:20:00 Starting - Launching requested ML instances..

In [14]:
predictor2 = estimator2.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge')

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-ap-southeast-2-611188727347/tensorflow-training-2020-11-05-04-56-30-245/output/model.tar.gz.

In [ ]:
sagemaker.Session().delete_endpoint(predictor2.endpoint)